## Browser Automation and Scraping: CBP Wait Times

### The Objective:
#### Web scraping and data formatting are two of the most fundamental skills taught at Columbia University's [Lede Program](https://ledeprogram.com/) for data journalism. 
#### ...and properly presenting Python-based code via publishing tools like Quarto is another important objective. 


### The Tools and Processes: 
#### Scraping the US [Customs and Border Protection site](https://awt.cbp.gov/), obtaining information, formatting that information, feeding it to a chart-making platform -- and making it automatic. 
1. Browser automation implemented with Playwright
2. Web scraping performed with Beautiful Soup4
3. Charts automatically generated by Datawrapper...
4. Same process repeated every day at 1300 GMT thanks to GitHub Actions



#### import all relevant libraries and modules: 

In [1]:
from playwright.async_api import async_playwright
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime
import time
import json
import csv
import numpy as np
import re
import lxml.html

#### Use Playwright to launch the Chromium browser

In [2]:
playwright = await async_playwright().start()

In [3]:
browser = await playwright.chromium.launch(headless = True)

In [4]:
page = await browser.new_page()

#### ... and open the CBP webpage

In [5]:
await page.goto("https://awt.cbp.gov/")
time.sleep(6)

### Use Playwright to pick an airport -- JFK in our case: 

In [6]:
await page.get_by_label("Select Your Airport/Terminal").select_option("JFK")

['JFK']

#### and also have Playwright click on the "Create Report" button: 

In [7]:
await page.get_by_role("button", name="Create Report").click()
time.sleep(10)

In [8]:
page = await page.content()
tables = pd.read_html(page)
tables[0]

Airport               Terminal      Date         Hour       U.S. Citizen  \
   Airport               Terminal      Date         Hour       U.S. Citizen   
   Airport               Terminal      Date         Hour Average Wait  Time   
0      JFK             Terminal 1  8/6/2023  0700 - 0800                 25   
1      JFK             Terminal 1  8/6/2023  0800 - 0900                 24   
2      JFK             Terminal 1  8/6/2023  0900 - 1000                 11   
3      JFK             Terminal 1  8/6/2023  1000 - 1100                 12   
4      JFK             Terminal 1  8/6/2023  1100 - 1200                 23   
..     ...                    ...       ...          ...                ...   
78     JFK  Terminal 8 (American)  8/6/2023  1800 - 1900                  9   
79     JFK  Terminal 8 (American)  8/6/2023  1900 - 2000                 21   
80     JFK  Terminal 8 (American)  8/6/2023  2000 - 2100                 13   
81     JFK  Terminal 8 (American)  8/6/2023  2100 - 2200                 11   
82     JFK  Terminal 8 (American)  8/6/2023  2200 - 2300                 12   

                    Non U.S. Citizen                               All  \
                    Non U.S. Citizen                        Wait Times   
   Max Wait  Time Average Wait  Time Max Wait  Time Average Wait  Time   
0              51                 33             52                 29   
1              45                 27             46                 26   
2              22                 12             23                 11   
3              27                 16             29                 14   
4              50                 39             79                 32   
..            ...                ...            ...                ...   
78             52                 20             56                 15   
79             78                 45             90                 33   
80             47                 34             65                 22   
81             37                 16             37                 13   
82             41                 26             47                 19   

                   ...                                                       \
                   ... Number Of Passengers Time Interval                     
   Max Wait  Time  ...                              16-30 31-45 46-60 61-90   
0              52  ...                                101    89    37     0   
1              46  ...                                138   111     4     0   
2              23  ...                                 74     0     0     0   
3              29  ...                                136     0     0     0   
4              79  ...                                523   315   167   123   
..            ...  ...                                ...   ...   ...   ...   
78             56  ...                                100    64    24     0   
79             90  ...                                492   231    80   221   
80             65  ...                                195    82    76    26   
81             37  ...                                206    33     0     0   
82             47  ...                                280   175    10     0   

                                                            \
                   Unnamed: 17_level_1 Unnamed: 18_level_1   
   91-120 120 plus            Excluded               Total   
0       0        0                   6                 277   
1       0        0                   8                 323   
2       0        0                   9                 273   
3       0        0                  10                 305   
4       0        0                  45                1340   
..    ...      ...                 ...                 ...   
78      0        0                  14                 604   
79      0        0                  40                1312   
80      0        0                  20                 744   
81      

In [ ]:
/html/body/div[3]/div[1]/span 

In [9]:
table = tables[0]

In [10]:
table.head()

Airport    Terminal      Date         Hour       U.S. Citizen  \
  Airport    Terminal      Date         Hour       U.S. Citizen   
  Airport    Terminal      Date         Hour Average Wait  Time   
0     JFK  Terminal 1  8/6/2023  0700 - 0800                 25   
1     JFK  Terminal 1  8/6/2023  0800 - 0900                 24   
2     JFK  Terminal 1  8/6/2023  0900 - 1000                 11   
3     JFK  Terminal 1  8/6/2023  1000 - 1100                 12   
4     JFK  Terminal 1  8/6/2023  1100 - 1200                 23   

                   Non U.S. Citizen                               All  \
                   Non U.S. Citizen                        Wait Times   
  Max Wait  Time Average Wait  Time Max Wait  Time Average Wait  Time   
0             51                 33             52                 29   
1             45                 27             46                 26   
2             22                 12             23                 11   
3             27                 16             29                 14   
4             50                 39             79                 32   

                  ...                                                       \
                  ... Number Of Passengers Time Interval                     
  Max Wait  Time  ...                              16-30 31-45 46-60 61-90   
0             52  ...                                101    89    37     0   
1             46  ...                                138   111     4     0   
2             23  ...                                 74     0     0     0   
3             29  ...                                136     0     0     0   
4             79  ...                                523   315   167   123   

                                                                               \
                  Unnamed: 17_level_1 Unnamed: 18_level_1 Unnamed: 19_level_1   
  91-120 120 plus            Excluded               Total             Flights   
0      0        0                   6                 277                   1   
1      0        0                   8                 323                   1   
2      0        0                   9                 273                   1   
3      0        0                  10                 305                   1   
4      0        0                  45                1340                   4   

                       
  Unnamed: 20_level_1  
  Unnamed: 20_level_2  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  

[5 rows x 21 columns]

In [13]:
# clr = table.select("clr_5")
data_date = 
# print(clr)

AttributeError: 'DataFrame' object has no attribute 'select'

In [17]:
table.columns.values

array([('Airport', 'Airport', 'Airport'),
       ('Terminal', 'Terminal', 'Terminal'), ('Date', 'Date', 'Date'),
       ('Hour', 'Hour', 'Hour'),
       ('U.S. Citizen', 'U.S. Citizen', 'Average Wait  Time'),
       ('U.S. Citizen', 'U.S. Citizen', 'Max Wait  Time'),
       ('Non U.S. Citizen', 'Non U.S. Citizen', 'Average Wait  Time'),
       ('Non U.S. Citizen', 'Non U.S. Citizen', 'Max Wait  Time'),
       ('All', 'Wait Times', 'Average Wait  Time'),
       ('All', 'Wait Times', 'Max Wait  Time'),
       ('All', 'Number Of Passengers Time Interval', '0-15'),
       ('All', 'Number Of Passengers Time Interval', '16-30'),
       ('All', 'Number Of Passengers Time Interval', '31-45'),
       ('All', 'Number Of Passengers Time Interval', '46-60'),
       ('All', 'Number Of Passengers Time Interval', '61-90'),
       ('All', 'Number Of Passengers Time Interval', '91-120'),
       ('All', 'Number Of Passengers Time Interval', '120 plus'),
       ('All', 'Unnamed: 17_level_1', 'Excluded'),

### We clean up the columns and consolidate in one row

In [18]:
table.columns = ['_'.join(col) for col in table.columns.values]

In [19]:
table.rename(columns = {'Airport_Airport_Airport': 'Airport', 'Terminal_Terminal_Terminal':'Terminal', 'Date_Date_Date':'date', 'Hour_Hour_Hour':'hour' }, inplace = True)

In [20]:
table.rename(columns = {'U.S. Citizen_U.S. Citizen_Average Wait  Time': 'US Citizen Avg', 'U.S. Citizen_U.S. Citizen_Max Wait  Time':'US Citizen Max', 'Non U.S. Citizen_Non U.S. Citizen_Average Wait  Time':'Non-Citizen Avg', 'Non U.S. Citizen_Non U.S. Citizen_Max Wait  Time': 'Non-Citizen Max' }, inplace = True)

In [21]:
table.rename(columns = {'All_Wait Times_Average Wait  Time': 'Avg Wait Time', 'All_Wait Times_Max Wait  Time':'Max Wait Time', 'All_Unnamed: 18_level_1_Total':'Total Passengers', 'All_Unnamed: 19_level_1_Flights': 'Total Flights' }, inplace = True)

In [22]:
table = table[['Airport', 'Terminal', 'date', 'hour','US Citizen Avg', 'US Citizen Max', 'Non-Citizen Avg','Non-Citizen Max','Avg Wait Time', 'Max Wait Time','Total Passengers','Total Flights' ]]

In [ ]:
#NEW NOTEBOOK

In [ ]:
#as soon as table is pretty, "jfk_today.csv"

In [ ]:
#NEW NOTEBOOK

In [ ]:
# take from today.csv file

In [ ]:
df =  pd read ==> append to large "totaljfk.csv"

In [26]:
term1 = table[table["Terminal"].str.contains("1")].reset_index(inplace=False)

In [53]:
term1 = term1.drop('index', axis =1)

In [28]:
reduced1 = term1[['hour','US Citizen Avg', 'Non-Citizen Avg']]

In [54]:
reducedtransposed1 = reduced1.transpose()

In [31]:
reducedtransposed1.columns = reducedtransposed1.iloc[0]

In [32]:
reducedtransposed1 = reducedtransposed1.drop(index= "hour")

In [33]:
reducedtransposed1 

hour,0900 - 1000,1100 - 1200,1200 - 1300,1300 - 1400,1400 - 1500,1500 - 1600,1600 - 1700,1700 - 1800,1800 - 1900,1900 - 2000,2000 - 2100,2100 - 2200,2200 - 2300,2300 - 0000
US Citizen Avg,31,38,41,27,53,49,44,21,28,25,26,33,31,17
Non-Citizen Avg,41,86,58,77,100,114,110,98,47,34,66,65,36,17


In [34]:
reducedtransposed1.to_csv("term1_us_nonus.csv")

### Terminal 4:

In [36]:
term4 = table[table["Terminal"].str.contains("4")].reset_index(inplace=False)

In [37]:
term4 = term4.drop('index', axis =1)
reduced4 = term4[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed4 = reduced4.transpose()
reducedtransposed4.columns = reducedtransposed4.iloc[0]
reducedtransposed4 = reducedtransposed4.drop(index= "hour")

In [43]:
reducedtransposed4.to_csv("term4_us_nonus.csv")

### Terminal 5: 

In [40]:
term5 = table[table["Terminal"].str.contains("5")].reset_index(inplace=False)

In [41]:
term5 = term5.drop('index', axis =1)
reduced5 = term5[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed5 = reduced5.transpose()
reducedtransposed5.columns = reducedtransposed5.iloc[0]
reducedtransposed5 = reducedtransposed5.drop(index= "hour")

In [45]:
reducedtransposed5.to_csv("term5_us_nonus.csv")

### Terminal 7: 

In [47]:
term7 = table[table["Terminal"].str.contains("7")].reset_index(inplace=False)

In [48]:
term7 = term7.drop('index', axis =1)
reduced7 = term7[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed7 = reduced7.transpose()
reducedtransposed7.columns = reducedtransposed7.iloc[0]
reducedtransposed7 = reducedtransposed7.drop(index= "hour")

In [49]:
reducedtransposed7.to_csv("term7_us_nonus.csv")

### Terminal 8: 

In [55]:
term8 = table[table["Terminal"].str.contains("8")].reset_index(inplace=False)

In [56]:
reduced8 = term8[['hour','US Citizen Avg', 'Non-Citizen Avg']]
reducedtransposed8 = reduced8.transpose()
reducedtransposed8.columns = reducedtransposed8.iloc[0]
reducedtransposed8 = reducedtransposed8.drop(index= "hour")

In [57]:
reducedtransposed8.to_csv("term8_us_nonus.csv")

In [58]:
ls

README.md              original_cbp.ipynb     term7_us_nonus.csv
action.yml             term1_us_nonus.csv     term8_us_nonus.csv
cbp1.ipynb             term4_us_nonus.csv
jfkterm8_us_nonus.csv  term5_us_nonus.csv


In [59]:
await browser.close()